In [3]:
import os
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

class ImageClassifier:
    def __init__(self, base_path, image_size=(224, 224), batch_size=32, epochs=20, num_classes=3):
        self.base_path = base_path
        self.image_size = image_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.num_classes = num_classes
        self.train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            validation_split=0.2
        )

    def build_model(self):
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=self.image_size + (3,))
        for layer in base_model.layers:
            layer.trainable = False
        
        x = Flatten()(base_model.output)
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(128, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        predictions = Dense(self.num_classes, activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        return model

    def train_and_evaluate(self):
        train_generator = self.train_datagen.flow_from_directory(
            self.base_path,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='training'
        )

        validation_generator = self.train_datagen.flow_from_directory(
            self.base_path,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='validation'
        )

        model = self.build_model()

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

        model.fit(
            train_generator,
            epochs=self.epochs,
            validation_data=validation_generator,
            callbacks=[early_stopping, reduce_lr]
        )

        # Evaluate the model
        test_loss, test_accuracy = model.evaluate(validation_generator)
        print(f"Test accuracy: {test_accuracy:.2f}")

if __name__ == "__main__":
    base_path = "./processed_data"
    classifier = ImageClassifier(base_path)
    classifier.train_and_evaluate()


Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Epoch 1/20


C:\Users\mamir\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 247s 3s/step - accuracy: 0.5781 - loss: 1.1774 - val_accuracy: 0.6383 - val_loss: 1.0707 - learning_rate: 0.0010
Epoch 2/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 227s 3s/step - accuracy: 0.7874 - loss: 0.5789 - val_accuracy: 0.8000 - val_loss: 0.6074 - learning_rate: 0.0010
Epoch 3/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 233s 3s/step - accuracy: 0.8458 - loss: 0.4252 - val_accuracy: 0.8150 - val_loss: 0.4927 - learning_rate: 0.0010
Epoch 4/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 225s 3s/step - accuracy: 0.8705 - loss: 0.3413 - val_accuracy: 0.8500 - val_loss: 0.4089 - learning_rate: 0.0010
Epoch 5/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 225s 3s/step - accuracy: 0.8673 - loss: 0.3794 - val_accuracy: 0.4567 - val_loss: 2.5073 - learning_rate: 0.0010
Epoch 6/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 225s 3s/step - accuracy: 0.8815 - loss: 0.2897 - val_accuracy: 0.5683 - val_loss: 1.9009 - learning_rate: 0.0010
Epoch 7/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 227s 3s/step - accuracy: 0.8737 - loss: 0.3122 - val_accuracy: 0.82